In [1]:
import os
import sys

import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns

sys.path.append('./lib/2 Eye Tracking')

from preprocess import *

%matplotlib inline

In [2]:
e_list = [load_eyetracker_file(e) for e in eyet_files]     # lists of dataframes containing respective data
m_list = [load_mouse_file(m) for m in mous_files]
g_list = [load_game_file(g) for g in mous_files]

t = [
    make_tidy(e_list[i], m_list[i], g_list[i]) 
    for i in range(len(e_list))
]                                                          # create tidy dfs per subject along timestamp index

mpivs = []                                                 # holding lists for pivoted histograms
epivs = []

for i in range(len(m_list)):                               # for each subject
    
    # MOUSE HISTOGRAMS
    m_list[i], mpvt = mouse_hist(m_list[i], g_list[i])
    mpivs.append(mpvt)
    
    # EYE HISTOGRAMS
    
    e_list[i], epvt = eye_hist(e_list[i], g_list[i])
    epivs.append(epvt)

/Users/gianni/anaconda/lib/python3.5/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
./lib/2 Eye Tracking/preprocess.py:264: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  gp['turnstart'] = gp.index - gp['rt']
./lib/2 Eye Tracking/preprocess.py:265: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexi

# Statistics
## Check eye-mouse correlations

In [ ]:
figrows = len(e_list)                                       # n rows == n subjects
fig, axes = plt.subplots(figrows, 2, figsize=(10, 40))      # initialize figure
scatterkws = {'linestyle': 'none', 'marker': 'o', 'markersize': 1, 'alpha': .1}
                                                            # set kws for scatter plot
xrs = []                                                    # initialize lists for x and y r values (convert to numpy?)
yrs = []
for subject, ee in enumerate(e_list):                       # for every subject
    for j, dim in enumerate(['x', 'y']):                    # for x and y dimensions      
        ul = 4 if dim=='y' else 9
        t_ = t[subject]                                     # select subject data
        nanfilter = pd.notnull(t_['eye' + dim]) & pd.notnull(t_['mou' + dim])
                                                            # filter records without mouse and eye vals
        trialfilter = t[subject]['task']=='playing'         # filter convenience records
        tt = t[subject].loc[nanfilter & trialfilter]        # apply filters
        eyebounds = (tt['eye' + dim] >= 0) & (tt['eye' + dim] <= ul)
        mousebounds = (tt['mou' + dim] >= 0) & (tt['mou' + dim] <= ul)
                                                            # filter eye  and mousecoordinates off board
        tt = tt.loc[eyebounds & mousebounds]                # apply eyebounds and mousebounds filters
        lr = stats.linregress(tt['eye' + dim], tt['mou' + dim])
                                                            # regress eye coordinate against mouse coordinate
        
        if dim == 'x':                                        # save regression coefficients
            xrs.append(lr.rvalue)
        else:
            yrs.append(lr.rvalue)
             
        ax = axes[subject, j]                               # select axes
        linex = [-2, 11]
        liney = [-2*lr.slope + lr.intercept, 11*lr.slope + lr.intercept]
                                                            # x and y coordinates for plotting regression line
        ax.plot(linex, liney, color='black')                # plot regression line
        ax.plot(tt['eye' + dim], tt['mou' + dim], **scatterkws)
                                                            # scatter eye coordinates against mouse coordinates

        
plt.setp(axes[:, 0], xlim=[-1, 10], ylim=[-1, 10])          # plot formatting
plt.setp(axes[:, 1], xlim=[-1, 5], ylim=[-1, 5])
plt.setp(axes[:-1, :], xticklabels=[])
plt.setp(axes[-1, 0], xlabel='Eye Pos (board coord.)')
plt.setp(axes[4, 0], ylabel='Mouse Pos (board coord.)')
plt.setp(axes[0, 0], title='X')
plt.setp(axes[0, 1], title='Y')

sns.despine()
plt.tight_layout()

In [ ]:
fig, axes = plt.subplots(1, 1, figsize=(5, 5))
scatterkws['markersize'] = 10
scatterkws['alpha'] = .9
axes.plot(xrs, yrs, **scatterkws)
plt.setp(axes, xlabel='X dim Pearson R', ylabel='Y dim Pearson R', xlim=[0, 1], ylim=[0, 1])

sns.despine()
plt.tight_layout()

In [ ]:
subject = 1
m = mpivs[subject]
e = epivs[subject]
mtidy = m_list[subject]
etidy = e_list[subject]
locs = list(range(36)) + [999]

h = m[locs].values / m[locs].values.sum(axis=1)[:, np.newaxis]
v = m[locs[:-1]].values / m[locs[:-1]].values.sum(axis=1)[:, np.newaxis]
x = m['zet'].values

he = e[locs].values / e[locs].values.sum(axis=1)[:, np.newaxis]
ve = e[locs[:-1]].values / e[locs[:-1]].values.sum(axis=1)[:, np.newaxis]

probe = 2
v[probe, x[probe]]

In [ ]:
def position_str_to_tensor(pstr):
    p = np.array(list(pstr)).astype(int).reshape([4, 9])[::-1, :]
    return p

def show_pieces(position=('0'*36, '0'*36), ax=None, **scatterkws):
    if ax is None:
        ax = plt.gca()
        
    bp, wp = [position_str_to_tensor(pstr) for pstr in position]
    ax.plot(*np.where(bp)[::-1], color='black', **scatterkws)
    ax.plot(*np.where(wp)[::-1], color='white', **scatterkws)
    
    plt.setp(
        ax, 
        xlim=[-.5, 8.5], ylim=[-.5, 3.5], 
        xticks=np.arange(-.5, 9, 1), yticks=np.arange(-.5, 4, 1),
        xticklabels=[], yticklabels=[],
        aspect=1
    )
    
    return ax

def show_zet(zet, ax, **scatterkws):
    x = zet % 9
    y = 3 - (zet // 9)
    ax.plot(x, y, **scatterkws)
    return None

def show_path(path, ax):
    x = path[:, 0] - .5
    y = path[:, 1] - .5
    ax.plot(x, 3 - y)
    ax.plot(x[0], 3 - y[0], color='green', linestyle='none', marker='o')
    ax.plot(x[-1], 3 - y[-1], color='red', linestyle='none', marker='o')
    return None

def show_eye_path(path, ax):
    x = path[:, 0] - .5
    y = path[:, 1] - .5
    ax.plot(x, 3 - y, color='purple')
    ax.plot(x[0], 3 - y[0], color='green', linestyle='none', marker='o')
    ax.plot(x[-2], 3 - y[-2], color='red', linestyle='none', marker='o')
    return None

scatterkws = {
    'marker': 'o', 'markersize': 25, 'markeredgecolor': 'black', 'markeredgewidth': 1, 
    'linestyle': 'none'
}

fig, axes = plt.subplots(8, 4, figsize=(24,24))

subject = 2
m = mpivs[subject]
mtidy = m_list[subject]
etidy = e_list[subject]

af = axes.flatten()
for i, turn in enumerate(m.index.values[:32]):
    ax = af[i]
    d = m.loc[turn]
    md = mtidy.loc[mtidy['turn']==turn]
    ed = etidy.loc[etidy['turn'].astype(int)==turn]

    show_pieces(d[['bp', 'wp']].values, ax=ax, **scatterkws)
    show_zet(d['zet'], ax=ax, **scatterkws)
    show_path(md[['xtile', 'ytile']].values, ax=ax)
    if len(ed) > 0:
        show_eye_path(ed[['transx', 'transy']].values, ax=ax)

sns.despine()
plt.tight_layout()